In [1]:
import pandas as pd

In [105]:
from lxml import html,etree
import requests
import pymongo
from requests import ConnectionError
import time
import re

In [3]:
from pymongo import MongoClient
client = MongoClient()
db = client.test_database
coll = db.test_collection

In [223]:
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,
 encoding='latin-1')

#items.head(8)

In [224]:
column_list = ['release_date','director','genre','creator','actor_list','_id']
reqd_column = ['movie id','movie title','IMDb URL']
reqd_movie_data = items[reqd_column]
#reqd_movie_data.head(8)
type(reqd_movie_data.loc[529])

pandas.core.series.Series

In [225]:
for row in reqd_movie_data[1642:].iterrows():
    url = row[1]['IMDb URL']
    m = re.search('imdb\-title\-\d+',url)
    if m:
        url = 'http://us.imdb.com/M/title-exact?'+row[1]['movie title'].replace(' ','%20')
    
    try:
        
        print url
        try:
            page = requests.get(url,timeout = 8)
        except ConnectionError:
            print 'Sending in 3 seconds'
            time.sleep(3)
            page = requests.get(url,timeout = 8)
        tree = html.fromstring(page.content)
        director = tree.xpath('//span[@itemprop="director"]//span[@itemprop="name"]/text()')
        genre = tree.xpath('//div[@itemprop="genre"]/a/text()')
        creator = tree.xpath('//span[@itemprop="creator"]//span[@itemprop="name"]/text()')
        actor_list = tree.xpath('//table[@class="cast_list"]//span[@itemprop="name"]/text()')[:5] 
        _id = tree.xpath('//meta[@property="pageId"]/@content')[0]
        release_date = tree.xpath('//meta[@itemprop="datePublished"]/@content')[0]
    except IndexError:
        if not ' in development ' in tree.xpath('//tr[@class="findResult odd"][1]/td[@class="result_text"]/text()') :
            print 'a'
            url = tree.xpath('//tr[@class="findResult odd"][1]/td[@class="result_text"]/a/@href')[0]
            
            print url
        else:
            url = tree.xpath('//tr[@class="findResult even"][1]/td[last()]/a/@href')[0]
            print 'b'
            print url
        u1 = url.rpartition('/')[0].rpartition('/')[-1]
        url = 'http://www.imdb.com/title/'+u1
        print 'new url:',url
        try:
            page = requests.get(url)
        except ConnectionError:
            print 'Sending in 3 seconds'
            time.sleep(5)
            page = requests.get(url)
        tree = html.fromstring(page.content)
        director = tree.xpath('//span[@itemprop="director"]//span[@itemprop="name"]/text()')
        genre = tree.xpath('//div[@itemprop="genre"]/a/text()')
        creator = tree.xpath('//span[@itemprop="creator"]//span[@itemprop="name"]/text()')
        actor_list = tree.xpath('//table[@class="cast_list"]//span[@itemprop="name"]/text()')[:5] 
        _id = tree.xpath('//meta[@property="pageId"]/@content')[0]
        release_date = tree.xpath('//meta[@itemprop="datePublished"]/@content')[0]
    
    new_elem = dict([(i,eval(i)) for i in column_list])
    #print new_elem
    new_doc = dict(row[1].to_dict().items() + new_elem.items())
    #print new_doc
    try:
        coll.insert_one(new_doc)
    except pymongo.errors.DuplicateKeyError:
        print 'Duplicate Key Not Inserting'+new_doc['movie title']
        continue
    print 'done'

http://us.imdb.com/M/title-exact?Angel%20Baby%20(1995)
done
http://us.imdb.com/M/title-exact?Sudden%20Manhattan%20%281996%29
done
http://us.imdb.com/M/title-exact?Butcher%20Boy,%20The%20(1998)
a
/title/tt0118804/?ref_=fn_tt_tt_1
new url: http://www.imdb.com/title/tt0118804
done
http://us.imdb.com/Title?Men+with+Guns+(1997/I)
done
http://us.imdb.com/Title?Hana-bi+(1997)
done
http://us.imdb.com/Title?Niagara,+Niagara+(1997)
done
http://us.imdb.com/Title?Big+One,+The+(1997)
a
/title/tt0124295/?ref_=fn_al_tt_1
new url: http://www.imdb.com/title/tt0124295
done
http://us.imdb.com/M/title-exact?Butcher%20Boy,%20The%20(1998)
a
/title/tt0118804/?ref_=fn_tt_tt_1
new url: http://www.imdb.com/title/tt0118804
Duplicate Key Not InsertingButcher Boy, The (1998)
http://us.imdb.com/Title?Spanish+Prisoner,+The+(1997)
a
/title/tt0120176/?ref_=fn_al_tt_1
new url: http://www.imdb.com/title/tt0120176
done
http://us.imdb.com/M/title-exact?Feng%20Yue%20%281996%29
a
/title/tt0116295/?ref_=fn_tt_tt_1
new url: h

In [226]:
print coll.find().sort('movie id',pymongo.DESCENDING)[0]
print coll.count()

{u'movie id': 1682, u'creator': [u'Hans-Ulrich Klenner', u'Walter Saxer', u'Sera Filmproduktion', u'Mol\xe9cule', u'Films A2'], u'actor_list': [u'Vittorio Mezzogiorno', u'Stefan Glowacz', u'Mathilda May', u'Al Waxman', u'Gunilla Karlzen'], u'release_date': u'1991-10-03', u'director': [u'Werner Herzog'], u'IMDb URL': u'http://us.imdb.com/M/title-exact?Schrei%20aus%20Stein%20(1991)', u'movie title': u'Scream of Stone (Schrei aus Stein) (1991)', u'genre': [u' Drama'], u'_id': u'tt0102855'}
1652


In [34]:
test_movie = {'_id':items.loc[4,'movie id'],
             'movie_title':items.loc[4,'movie title'],
             'release_date':items.loc[4,'release date'],
             'director':director,
             'genre':genre,
             'creator':creator,
             'actor_list':actor_list,
             'imdb_id':imdb_id}

In [19]:
coll.find_one()

{u'IMDb URL': u'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
 u'_id': u'tt0114709',
 u'actor_list': [u'Tom Hanks',
  u'Tim Allen',
  u'Don Rickles',
  u'Jim Varney',
  u'Wallace Shawn'],
 u'creator': [u'John Lasseter',
  u'Pete Docter',
  u'Pixar Animation Studios',
  u'Walt Disney Pictures'],
 u'director': [u'John Lasseter'],
 u'genre': [u' Animation', u' Adventure', u' Comedy', u' Family', u' Fantasy'],
 u'movie id': 1,
 u'movie title': u'Toy Story (1995)',
 u'release_date': u'1995-11-22'}

In [49]:
a = "'hfbf'"
x = 'a'
eval(x)

"'hfbf'"

In [77]:
column_list = ['release_date','director','genre','creator','actor_list','_id']
del_post = coll.delete_many({})
print del_post.deleted_count

25
